In [1]:
### Create a MLFlow model deploy it to a rest api


In [1]:
import keras

2025-04-20 03:32:07.737254: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Matplotlib is building the font cache; this may take a moment.


In [3]:
import numpy as np
import pandas as pd

In [6]:
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [9]:
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",sep=';',)

In [10]:
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [13]:
## Split the data into train,test,validation

train,test = train_test_split(data,test_size=0.2,random_state=0)

In [14]:
train_x = train.drop(['quality'],axis=1).values
train_y = train['quality'].values.ravel()
test_x = test.drop(['quality'],axis=1).values
test_y = test['quality'].values.ravel()

train_x,valid_x,train_y,valid_y = train_test_split(train_x,train_y,test_size=0.2,random_state=0)

In [16]:
signature = infer_signature(train_x,train_y)

In [17]:
def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):
    # Define model architecture
    mean=np.mean(train_x,axis=0)
    var = np.var(train_x,axis=0)

    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)

        ]
    )
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params['lr'],momentum=params['momentum']
    ),
    loss='mean_squared_error',
    metrics = [keras.metrics.RootMeanSquaredError()]    
    )


    ### train the ANN model with lr and momentum params with mlflow tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
            epochs=epochs,
            batch_size=32
        )
        eval_result = model.evaluate(valid_x,valid_y,batch_size=32)
        eval_rmse = eval_result[1]

        mlflow.log_params(params)
        mlflow.log_metric('eval_rmse',eval_rmse)

        mlflow.tensorflow.log_model(model,'model',signature=signature)

        return {'loss':eval_rmse,'status':STATUS_OK,'model':model}

In [18]:
def objective(params):
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x = valid_x,
        valid_y=valid_y,
        test_x = test_x,
        test_y = test_y,
    )
    return result

In [19]:
space = {
    "lr":hp.loguniform('lr',np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform('momentum',0.0,1.0)
}

In [25]:
mlflow.set_experiment("/wine-quality")

with mlflow.start_run():
    trials = Trials()
    best = fmin(
        fn = objective,
        space = space,
        algo = tpe.suggest,
        max_evals = 4,
        trials = trials
    )

    best_run = sorted(trials.results,key = lambda x:x['loss'])[0]

    mlflow.log_params(best)
    mlflow.log_metric("eval_score",best_run['loss'])
    mlflow.tensorflow.log_model(best_run['model'],'model',signature=signature)
    

    print(f"Best parameter: {best}")
    print(f"best eval rmse: {best_run['loss']}")


Epoch 1/3                                            

 1/98 ━━━━━━━━━━━━━━━━━━━━ 2:24 1s/step - loss: 38.8202 - root_mean_squared_error: 6.2306
 6/98 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 19.7900 - root_mean_squared_error: 4.3275
11/98 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 14.3321 - root_mean_squared_error: 3.6262
20/98 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 10.1397 - root_mean_squared_error: 3.0003 
28/98 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 8.2852 - root_mean_squared_error: 2.6885 
37/98 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 6.9794 - root_mean_squared_error: 2.4498
45/98 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 6.1779 - root_mean_squared_error: 2.2937
49/98 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 5.8569 - root_mean_squared_error: 2.2288
50/98 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 5.7832 - root_mean_squared_error: 2.2137
53/98 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 5.5754 - root_mean_squared_error: 2.1706
64/98 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 4.9591